<a href="https://colab.research.google.com/github/domingues100/IEEE---Water_Level/blob/main/Regress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# IMPORTS

In [ ]:
import os
from shutil import copy2
import csv
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
import statistics
from keras.layers import Lambda
import joblib
import tensorflow_hub as hub

# **KFOLD**

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/IC_Machine_Learning/instances_default.csv')

for ind in train_data.index:
  train_data.loc[ind, 'label'] = train_data.loc[ind, 'label'] - 3

Y = train_data[['label']]

In [ ]:
num_epochs = 100

idg = ImageDataGenerator(width_shift_range=0.3,
                         height_shift_range=0.3,
                         rotation_range=0.4,
                         zoom_range=0.4,
                         fill_mode='nearest',
                         horizontal_flip = True,
                         rescale=1./255)

In [ ]:
save_dir = #save_path
image_dir = #img dir ->remember to resize to 224x224

folder_old_path = #path to the folds used in class


fold_var = 1
n = Y.size


for i in range(1,6):
  training_data = pd.read_csv(f"{folder_old_path}/training_data{i}.csv")
  validation_data = pd.read_csv(f"{folder_old_path}o/validation_data{i}.csv")

  scaler = StandardScaler()
  scaler.fit(training_data["label"].to_numpy().reshape([-1,1]))

  validation_data["normalizado"] = scaler.transform(validation_data["label"].to_numpy().reshape([-1,1]))
  training_data["normalizado"] = scaler.transform(training_data["label"].to_numpy().reshape([-1,1]))

  #data generators - training and validating
  train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
						             x_col = "id", y_col = "normalizado",
						             class_mode = "raw", shuffle = True, batch_size=32)

  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir,
							           x_col = "id", y_col = "normalizado",
							           class_mode = "raw", shuffle = False, batch_size=32)


#Model
  base_model = keras.applications.ResNet50V2(
               include_top=False,
               weights='imagenet',
               input_shape=(224, 224, 3))

  base_model.trainable = False


  model = keras.Sequential([

    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation ='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1, activation='linear')
  ])

  # Optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

	# Compile Model
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy', 'mse'])

	# Fit Model
  history = model.fit(train_data_generator, epochs=num_epochs, validation_data=valid_data_generator) #validation_data=valid_data_generator

  # Save Model
  tf.keras.backend.clear_session()
  model.save(save_dir + f"/model{fold_var}.h5")
  joblib.dump(scaler, save_dir +"/scaler"+str(fold_var)) #saves the standart scaler to use for predictions

  fold_var += 1
  gc.collect()

Found 171 validated image filenames.
Found 43 validated image filenames.
Epoch 1/100
6/6 [==============================] - 8s 822ms/step - loss: 1.6045 - accuracy: 0.0000e+00 - mse: 1.6045 - val_loss: 0.7183 - val_accuracy: 0.0000e+00 - val_mse: 0.7183
Epoch 2/100
6/6 [==============================] - 4s 610ms/step - loss: 1.2208 - accuracy: 0.0000e+00 - mse: 1.2208 - val_loss: 0.8645 - val_accuracy: 0.0000e+00 - val_mse: 0.8645
Epoch 3/100
6/6 [==============================] - 4s 591ms/step - loss: 1.3113 - accuracy: 0.0000e+00 - mse: 1.3113 - val_loss: 0.5141 - val_accuracy: 0.0000e+00 - val_mse: 0.5141
Epoch 4/100
6/6 [==============================] - 3s 571ms/step - loss: 0.9866 - accuracy: 0.0000e+00 - mse: 0.9866 - val_loss: 0.5490 - val_accuracy: 0.0000e+00 - val_mse: 0.5490
Epoch 5/100
6/6 [==============================] - 4s 594ms/step - loss: 0.7947 - accuracy: 0.0000e+00 - mse: 0.7947 - val_loss: 0.5997 - val_accuracy: 0.0000e+00 - val_mse: 0.5997
Epoch 6/100
6/6 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 171 validated image filenames.
Found 43 validated image filenames.
Epoch 1/100
6/6 [==============================] - 8s 905ms/step - loss: 2.0611 - accuracy: 0.0000e+00 - mse: 2.0611 - val_loss: 1.2722 - val_accuracy: 0.0000e+00 - val_mse: 1.2722
Epoch 2/100
6/6 [==============================] - 4s 614ms/step - loss: 1.6036 - accuracy: 0.0000e+00 - mse: 1.6036 - val_loss: 0.8125 - val_accuracy: 0.0000e+00 - val_mse: 0.8125
Epoch 3/100
6/6 [==============================] - 4s 610ms/step - loss: 1.2168 - accuracy: 0.0000e+00 - mse: 1.2168 - val_loss: 0.7511 - val_accuracy: 0.0000e+00 - val_mse: 0.7511
Epoch 4/100
6/6 [==============================] - 4s 622ms/step - loss: 1.0005 - accuracy: 0.0000e+00 - mse: 1.0005 - val_loss: 0.7444 - val_accuracy: 0.0000e+00 - val_mse: 0.7444
Epoch 5/100
6/6 [==============================] - 4s 696ms/step - loss: 0.8997 - accuracy: 0.0000e+00 - mse: 0.8997 - val_loss: 0.7136 - val_accuracy: 0.0000e+00 - val_mse: 0.7136
Epoch 6/100
6/6 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 171 validated image filenames.
Found 43 validated image filenames.
Epoch 1/100
6/6 [==============================] - 8s 938ms/step - loss: 1.9091 - accuracy: 0.0000e+00 - mse: 1.9091 - val_loss: 1.2286 - val_accuracy: 0.0000e+00 - val_mse: 1.2286
Epoch 2/100
6/6 [==============================] - 4s 610ms/step - loss: 1.4212 - accuracy: 0.0000e+00 - mse: 1.4212 - val_loss: 0.8458 - val_accuracy: 0.0000e+00 - val_mse: 0.8458
Epoch 3/100
6/6 [==============================] - 4s 591ms/step - loss: 1.1545 - accuracy: 0.0000e+00 - mse: 1.1545 - val_loss: 0.8213 - val_accuracy: 0.0000e+00 - val_mse: 0.8213
Epoch 4/100
6/6 [==============================] - 4s 599ms/step - loss: 1.1013 - accuracy: 0.0000e+00 - mse: 1.1013 - val_loss: 0.6225 - val_accuracy: 0.0000e+00 - val_mse: 0.6225
Epoch 5/100
6/6 [==============================] - 4s 605ms/step - loss: 0.9835 - accuracy: 0.0000e+00 - mse: 0.9835 - val_loss: 0.4350 - val_accuracy: 0.0000e+00 - val_mse: 0.4350
Epoch 6/100
6/6 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 171 validated image filenames.
Found 43 validated image filenames.
Epoch 1/100
6/6 [==============================] - 8s 896ms/step - loss: 2.1643 - accuracy: 0.0000e+00 - mse: 2.1643 - val_loss: 1.1953 - val_accuracy: 0.0000e+00 - val_mse: 1.1953
Epoch 2/100
6/6 [==============================] - 4s 627ms/step - loss: 1.5906 - accuracy: 0.0000e+00 - mse: 1.5906 - val_loss: 1.0079 - val_accuracy: 0.0000e+00 - val_mse: 1.0079
Epoch 3/100
6/6 [==============================] - 4s 596ms/step - loss: 1.7658 - accuracy: 0.0000e+00 - mse: 1.7658 - val_loss: 1.0547 - val_accuracy: 0.0000e+00 - val_mse: 1.0547
Epoch 4/100
6/6 [==============================] - 4s 611ms/step - loss: 1.4042 - accuracy: 0.0000e+00 - mse: 1.4042 - val_loss: 0.7938 - val_accuracy: 0.0000e+00 - val_mse: 0.7938
Epoch 5/100
6/6 [==============================] - 4s 578ms/step - loss: 0.8638 - accuracy: 0.0000e+00 - mse: 0.8638 - val_loss: 0.6121 - val_accuracy: 0.0000e+00 - val_mse: 0.6121
Epoch 6/100
6/6 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 172 validated image filenames.
Found 42 validated image filenames.
Epoch 1/100
6/6 [==============================] - 9s 743ms/step - loss: 1.7955 - accuracy: 0.0000e+00 - mse: 1.7955 - val_loss: 1.1344 - val_accuracy: 0.0000e+00 - val_mse: 1.1344
Epoch 2/100
6/6 [==============================] - 4s 585ms/step - loss: 1.3334 - accuracy: 0.0000e+00 - mse: 1.3334 - val_loss: 0.7840 - val_accuracy: 0.0000e+00 - val_mse: 0.7840
Epoch 3/100
6/6 [==============================] - 4s 595ms/step - loss: 1.1494 - accuracy: 0.0000e+00 - mse: 1.1494 - val_loss: 0.9247 - val_accuracy: 0.0000e+00 - val_mse: 0.9247
Epoch 4/100
6/6 [==============================] - 3s 567ms/step - loss: 1.0149 - accuracy: 0.0000e+00 - mse: 1.0149 - val_loss: 1.0032 - val_accuracy: 0.0000e+00 - val_mse: 1.0032
Epoch 5/100
6/6 [==============================] - 3s 576ms/step - loss: 0.9700 - accuracy: 0.0000e+00 - mse: 0.9700 - val_loss: 0.6187 - val_accuracy: 0.0000e+00 - val_mse: 0.6187
Epoch 6/100
6/6 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Load Model

Necessary run diretory cells and idg before



In [ ]:
from keras.models import load_model

base_model = keras.applications.ResNet50V2(
               include_top=False,
               weights='imagenet',
               input_shape=(224, 224, 3))

base_model.trainable = False

#   # Model Def
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation ='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1, activation='linear')
  ])


In [ ]:
save_dir =  #save_path
image_dir = #image path ->resized to 224
folder_old_path = #path to the folds used in class

var = 1
accuracy = []
k=0


for k in range(5):
  validation_data = pd.read_csv(f"{folder_old_path}/validation_data{k+1}.csv")
  scaler =joblib.load(save_dir +"/scaler"+str(k+1))

  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir,
							           x_col = "id", y_col = "label",
							           class_mode = "raw", shuffle = False, batch_size=32)



  model.load_weights(f"{save_dir}/model{k+1}.h5")
  pred = model.predict(valid_data_generator)
  pred = scaler.inverse_transform(pred)
  pred = np.squeeze(pred)

  for itens in validation_data:
    val_data = validation_data["label"] .values.tolist()


  val_data = list(map(int, val_data))

  k += 1

  dados = {"predicted_desnormalizado": pred, "labels": val_data}
  df = pd.DataFrame(dados)
  df.to_csv(save_dir+f"/df{k}.csv")